# Flight Preprocessing

In [1]:
%AddJar file:///home/jovyan/work/apps/Emiasd-Flight-Data-Analysis.jar

Starting download from file:///home/jovyan/work/apps/Emiasd-Flight-Data-Analysis.jar
Finished download of Emiasd-Flight-Data-Analysis.jar
Using cached version of Emiasd-Flight-Data-Analysis.jar


In [2]:
import org.apache.spark.sql.SparkSession
import com.flightdelay.config.{AppConfiguration, ConfigurationLoader}
import com.flightdelay.data.loaders.FlightDataLoader

//Env Configuration
val args: Array[String] = Array("jupyter")

val spark = SparkSession.builder()
  .config(sc.getConf)
  .getOrCreate()

// Rendre la session Spark implicite
implicit val session = spark
implicit val configuration: AppConfiguration = ConfigurationLoader.loadConfiguration(args)

// Cellule 4: Test
val flightData = FlightDataLoader.loadFromConfiguration()


[STEP 1][DataLoader] Flight Data Loading - Start

Loading from existing Parquet file:
  - Path: /home/jovyan/work/output/common/data/raw_flights.parquet
  - Loaded 6096762 records from Parquet (optimized)

Schema:
root
 |-- FL_DATE: date (nullable = true)
 |-- OP_CARRIER_AIRLINE_ID: integer (nullable = true)
 |-- OP_CARRIER_FL_NUM: integer (nullable = true)
 |-- ORIGIN_AIRPORT_ID: integer (nullable = true)
 |-- DEST_AIRPORT_ID: integer (nullable = true)
 |-- CRS_DEP_TIME: integer (nullable = true)
 |-- ARR_DELAY_NEW: double (nullable = true)
 |-- CANCELLED: integer (nullable = true)
 |-- DIVERTED: integer (nullable = true)
 |-- CRS_ELAPSED_TIME: double (nullable = true)
 |-- WEATHER_DELAY: double (nullable = true)
 |-- NAS_DELAY: double (nullable = true)



args = Array(jupyter)
spark = org.apache.spark.sql.SparkSession@654f036
session = org.apache.spark.sql.SparkSession@654f036
configuration = AppConfiguration(local,CommonConfig(42,DataConfig(/home/jovyan/work/data,FileConfig(/home/jovyan/work/data/FLIGHT-3Y/Flights/201201.csv),FileConfig(/home/jovyan/work/data/FLIGHT-3Y/Weather/201201hourly.txt),FileConfig(/home/jovyan/work/data/FLIGHT-3Y/wban_airport_timezone.csv)),OutputConfig(/home/jovyan/work/output,FileConfig(/home/jovyan/work/output/data),FileConf...


AppConfiguration(local,CommonConfig(42,DataConfig(/home/jovyan/work/data,FileConfig(/home/jovyan/work/data/FLIGHT-3Y/Flights/201201.csv),FileConfig(/home/jovyan/work/data/FLIGHT-3Y/Weather/201201hourly.txt),FileConfig(/home/jovyan/work/data/FLIGHT-3Y/wban_airport_timezone.csv)),OutputConfig(/home/jovyan/work/output,FileConfig(/home/jovyan/work/output/data),FileConf...

In [3]:
import com.flightdelay.data.DataPipeline

val flightProcessedData = DataPipeline.execute()


[DataPipeline] Complete Data Pipeline - Start

[Step 1/5] Loading raw flight data...

[STEP 1][DataLoader] Flight Data Loading - Start

Loading from existing Parquet file:
  - Path: /home/jovyan/work/output/common/data/raw_flights.parquet
  - Loaded 6096762 records from Parquet (optimized)

Schema:
root
 |-- FL_DATE: date (nullable = true)
 |-- OP_CARRIER_AIRLINE_ID: integer (nullable = true)
 |-- OP_CARRIER_FL_NUM: integer (nullable = true)
 |-- ORIGIN_AIRPORT_ID: integer (nullable = true)
 |-- DEST_AIRPORT_ID: integer (nullable = true)
 |-- CRS_DEP_TIME: integer (nullable = true)
 |-- ARR_DELAY_NEW: double (nullable = true)
 |-- CANCELLED: integer (nullable = true)
 |-- DIVERTED: integer (nullable = true)
 |-- CRS_ELAPSED_TIME: double (nullable = true)
 |-- WEATHER_DELAY: double (nullable = true)
 |-- NAS_DELAY: double (nullable = true)

[Step 1/5] Completed in 0.145s

[Step 2/5] Loading raw weather data...

[STEP 1][DataLoader] Weather Data Loading - Start

Loading from existing Pa

flightProcessedData = ([FL_DATE: date, OP_CARRIER_AIRLINE_ID: int ... 109 more fields],[WBAN: string, Date: date ... 82 more fields])


([FL_DATE: date, OP_CARRIER_AIRLINE_ID: int ... 109 more fields],[WBAN: string, Date: date ... 82 more fields])

In [8]:
val flightsPath = s"${configuration.common.output.basePath}/common/data/processed_flights.parquet"
val flightsDF = spark.read.parquet(flightsPath)
flightsDF.printSchema

root
 |-- FL_DATE: date (nullable = true)
 |-- OP_CARRIER_AIRLINE_ID: integer (nullable = true)
 |-- OP_CARRIER_FL_NUM: integer (nullable = true)
 |-- ORIGIN_AIRPORT_ID: integer (nullable = true)
 |-- DEST_AIRPORT_ID: integer (nullable = true)
 |-- CRS_DEP_TIME: integer (nullable = true)
 |-- ARR_DELAY_NEW: double (nullable = true)
 |-- CRS_ELAPSED_TIME: double (nullable = true)
 |-- WEATHER_DELAY: double (nullable = true)
 |-- NAS_DELAY: double (nullable = true)
 |-- ORIGIN_WBAN: string (nullable = true)
 |-- ORIGIN_TIMEZONE: integer (nullable = true)
 |-- DEST_WBAN: string (nullable = true)
 |-- DEST_TIMEZONE: integer (nullable = true)
 |-- UTC_CRS_DEP_TIME: string (nullable = true)
 |-- UTC_FL_DATE: date (nullable = true)
 |-- UTC_ARR_TIME: string (nullable = true)
 |-- UTC_ARR_DATE: date (nullable = true)
 |-- CRS_ARR_TIME: string (nullable = true)
 |-- CRS_ARR_DATE: date (nullable = true)
 |-- feature_arrival_hour: integer (nullable = true)
 |-- feature_utc_arrival_hour: integer (

flightsPath = /home/jovyan/work/output/common/data/processed_flights.parquet
flightsDF = [FL_DATE: date, OP_CARRIER_AIRLINE_ID: int ... 109 more fields]


[FL_DATE: date, OP_CARRIER_AIRLINE_ID: int ... 109 more fields]

In [21]:
import org.apache.spark.sql.functions._

var flightsColumns = Seq(
"FL_DATE",
"OP_CARRIER_AIRLINE_ID",
"OP_CARRIER_FL_NUM",
"CRS_DEP_TIME",
"ARR_DELAY_NEW",
//"feature_utc_departure_hour_rounded",
"label_is_delayed_15min"    
)

var selectedFlightsDF = flightsDF
  .select(flightsColumns.head, flightsColumns.tail: _*)
  .orderBy(col("FL_DATE"), col("OP_CARRIER_AIRLINE_ID"), col("OP_CARRIER_FL_NUM"), col("CRS_DEP_TIME"))

selectedFlightsDF.show(10)

+----------+---------------------+-----------------+------------+-------------+----------------------+
|   FL_DATE|OP_CARRIER_AIRLINE_ID|OP_CARRIER_FL_NUM|CRS_DEP_TIME|ARR_DELAY_NEW|label_is_delayed_15min|
+----------+---------------------+-----------------+------------+-------------+----------------------+
|2012-01-01|                19393|                1|        1220|          0.0|                     0|
|2012-01-01|                19393|                3|        1815|          6.0|                     0|
|2012-01-01|                19393|                4|         955|          0.0|                     0|
|2012-01-01|                19393|               18|         730|          8.0|                     0|
|2012-01-01|                19393|               18|        1010|         54.0|                     1|
|2012-01-01|                19393|               26|         855|         43.0|                     1|
|2012-01-01|                19393|               26|        1410|        

flightsColumns = List(FL_DATE, OP_CARRIER_AIRLINE_ID, OP_CARRIER_FL_NUM, CRS_DEP_TIME, ARR_DELAY_NEW, label_is_delayed_15min)
selectedFlightsDF = [FL_DATE: date, OP_CARRIER_AIRLINE_ID: int ... 4 more fields]


[FL_DATE: date, OP_CARRIER_AIRLINE_ID: int ... 4 more fields]

In [19]:
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.functions._
import org.apache.spark.sql.expressions.Window
import org.apache.spark.sql.DataFrame

object LateAircraftFeature {
  
  /**
   * Crée la feature 'Late Aircraft' qui capture le retard de l'avion
   * sur son vol précédent de la journée.
   * 
   * @param df DataFrame contenant les données de vols
   * @return DataFrame enrichi avec la colonne PREV_AIRCRAFT_ARR_DELAY
   */
  def createLateAircraftFeature(df: DataFrame): DataFrame = {
    
    // Étape 1: Créer un identifiant d'avion proxy
    // En l'absence de TAIL_NUM, on combine compagnie + numéro de vol
    val dfWithAircraftId = df.withColumn(
      "AIRCRAFT_ID",
      concat(
        col("OP_CARRIER_AIRLINE_ID").cast("string"),
        lit("_"),
        col("OP_CARRIER_FL_NUM").cast("string")
      )
    )
    
    // Étape 2: Définir la fenêtre de partitionnement et tri
    // On partitionne par avion ET date, puis on trie par heure de départ prévue
    val windowSpec = Window
      .partitionBy("AIRCRAFT_ID", "FL_DATE")
      .orderBy("CRS_DEP_TIME")
    
    // Étape 3: Utiliser lag() pour récupérer le retard du vol précédent
    // lag() est l'équivalent Spark de shift() en pandas
    val dfWithPrevDelay = dfWithAircraftId.withColumn(
      "PREV_AIRCRAFT_ARR_DELAY",
      lag("ARR_DELAY_NEW", 1).over(windowSpec)
    )
    
    // Étape 4: Remplacer les valeurs null par 0
    // Les valeurs null correspondent au premier vol de la journée pour cet avion
    val dfFinal = dfWithPrevDelay.withColumn(
      "PREV_AIRCRAFT_ARR_DELAY",
      coalesce(col("PREV_AIRCRAFT_ARR_DELAY"), lit(0.0))
    )
    
    // Étape 5 (optionnelle): Créer une feature binaire indiquant si l'avion était en retard
    val dfWithBinaryFeature = dfFinal.withColumn(
      "IS_PREV_AIRCRAFT_LATE",
      when(col("PREV_AIRCRAFT_ARR_DELAY") > 0, 1).otherwise(0)
    )
    
    dfWithBinaryFeature
  }
  
  /**
   * Version alternative si vous avez la colonne TAIL_NUM
   */
  def createLateAircraftFeatureWithTailNum(df: DataFrame): DataFrame = {
    
    val windowSpec = Window
      .partitionBy("TAIL_NUM", "FL_DATE")
      .orderBy("CRS_DEP_TIME")
    
    val dfWithPrevDelay = df.withColumn(
      "PREV_AIRCRAFT_ARR_DELAY",
      lag("ARR_DELAY_NEW", 1).over(windowSpec)
    )
    
    val dfFinal = dfWithPrevDelay.withColumn(
      "PREV_AIRCRAFT_ARR_DELAY",
      coalesce(col("PREV_AIRCRAFT_ARR_DELAY"), lit(0.0))
    )
    
    dfFinal
  }

}

defined object LateAircraftFeature


In [20]:
val enrichedFlightsDF = LateAircraftFeature.createLateAircraftFeature(flightsDF)

enrichedFlightsDF = [FL_DATE: date, OP_CARRIER_AIRLINE_ID: int ... 112 more fields]


[FL_DATE: date, OP_CARRIER_AIRLINE_ID: int ... 112 more fields]

In [23]:
import org.apache.spark.sql.functions._

var flightsColumns = Seq(
"FL_DATE",
"OP_CARRIER_AIRLINE_ID",
"OP_CARRIER_FL_NUM",
"CRS_DEP_TIME",
"CRS_ELAPSED_TIME",    
"ARR_DELAY_NEW",
"PREV_AIRCRAFT_ARR_DELAY",
"label_is_delayed_15min"    
)

var selectedFlightsDF = enrichedFlightsDF
  .select(flightsColumns.head, flightsColumns.tail: _*)
  .orderBy(col("FL_DATE"), col("OP_CARRIER_AIRLINE_ID"), col("OP_CARRIER_FL_NUM"), col("CRS_DEP_TIME"))

selectedFlightsDF.show(10)

+----------+---------------------+-----------------+------------+----------------+-------------+-----------------------+----------------------+
|   FL_DATE|OP_CARRIER_AIRLINE_ID|OP_CARRIER_FL_NUM|CRS_DEP_TIME|CRS_ELAPSED_TIME|ARR_DELAY_NEW|PREV_AIRCRAFT_ARR_DELAY|label_is_delayed_15min|
+----------+---------------------+-----------------+------------+----------------+-------------+-----------------------+----------------------+
|2012-01-01|                19393|                1|        1220|           145.0|          0.0|                    0.0|                     0|
|2012-01-01|                19393|                3|        1815|            80.0|          6.0|                    0.0|                     0|
|2012-01-01|                19393|                4|         955|           170.0|          0.0|                    0.0|                     0|
|2012-01-01|                19393|               18|         730|           130.0|          8.0|                    0.0|                

flightsColumns = List(FL_DATE, OP_CARRIER_AIRLINE_ID, OP_CARRIER_FL_NUM, CRS_DEP_TIME, CRS_ELAPSED_TIME, ARR_DELAY_NEW, PREV_AIRCRAFT_ARR_DELAY, label_is_delayed_15min)
selectedFlightsDF = [FL_DATE: date, OP_CARRIER_AIRLINE_ID: int ... 6 more fields]


[FL_DATE: date, OP_CARRIER_AIRLINE_ID: int ... 6 more fields]